In [66]:
from collections import defaultdict
from importlib import reload
from pathlib import Path
import sys

from sklearn.datasets import fetch_20newsgroups

sys.path.insert(0, str(Path.cwd().parents[0] / 'src')) 
import prep; reload(prep)

<module 'prep' from '/Users/chris/Desktop/cryptogram-solver/src/prep.py'>

In [547]:
prep.NgramKind

SyntaxError: invalid syntax (<ipython-input-547-163cad45abc4>, line 1)

In [ ]:
from string import ascii_lowercase
LETTERS = set(ascii_lowercase)

In [263]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=20180827)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=20180827)

In [531]:
reload(prep)
prep.Token(('a',), prep.NgramKind.char, 1) == prep.Token(('a',), prep.NgramKind.char, 1)

True

In [539]:
reload(prep)
tokenizer = prep.Tokenizer(char_ngram_range=(1, 6), word_ngram_range=(1, 1), vocab_size=1000000)
tokenizer.fit(news_train['data'][:250])
print(len(tokenizer.vocab))
print(tokenizer.totals)

100%|██████████| 250/250 [00:23<00:00, 10.58it/s]


85485
defaultdict(<class 'int'>, {(<NgramKind.word: 1>, 1): 82631, (<NgramKind.char: 2>, 1): 467854, (<NgramKind.char: 2>, 2): 396195, (<NgramKind.char: 2>, 3): 329212, (<NgramKind.char: 2>, 4): 268452, (<NgramKind.char: 2>, 5): 208407, (<NgramKind.char: 2>, 6): 153166})


In [540]:
doc2 = prep.Doc(news_test['data'][10])
# doc2 = prep.Doc(doc2.swap('a', 'b'))
# doc2.scramble(15)
doc2.text

'from: ez027993@dale.ucdavis.edu (gary built like villanueva huckabay)\nsubject: ron gant, stalling, and hirschbeck.\norganization: julio machado candlelight vigil society\nlines: 88\n\n(dale "seer" stephenson) writes:\n>while i can see why they want to cut down on the time spent walking around,\n>gant wasn\'t ordered back in the box "sooner", he was ordered back immediately.\n>as soon as he stepped out, hirschbeck told him to get back in the box.\n>now, gant doesn\'t take a lot of walks between pitches.  the only reason\n>he did then because he was *very* bothered by the call.  i expect his\n>concentration wasn\'t there yet, and in a crucial situation i imagine it\'s\n>best to be as calm as possible.  contributing factors would be gant\'s\n>bad day at the plate, bad year at the plate, and the braves long scoring\n>drought.\n\nand it is not hirschbeck\'s job to help gant with any of these difficulties.\nif gant can\'t gather his concentration for whatever reason, that just makes\nhim a

In [541]:
def solve_cryptogram(text, tokenizer, max_epochs=50, is_debug=True):
    doc = prep.Doc(text)
    best_nll = tokenizer.calc_nll(doc.text)
    best_swap = ('a', 'a')
    epoch = 0
    improving = True
    while improving and epoch < max_epochs:
        improving = False
        print('*', end='')
        seen = set()
        for letter1 in doc.letters:
            for letter2 in LETTERS - set(letter1):
                swap = tuple(sorted([letter1, letter2]))
                if swap in seen:
                    continue
                seen.add(swap)
                new_text = doc.swap(*swap)
                nll = tokenizer.calc_nll(new_text)
                if nll < best_nll:
                    best_swap = swap
                    best_nll = nll
                    improving = True
        doc = prep.Doc(doc.swap(*best_swap))
        epoch += 1
        print(f'{doc.text}')
        print(f'best of epoch: {best_nll:0.5g}')
    print(f'\nfinal best ({epoch} epochs): {best_nll:0.5g}')
    return doc.text

# import random
# from math import exp

# def softmax(options, temp=1):
#     nums = [exp(x / temp) for x in options.values()]
#     denom = sum(nums)
#     choice_probs = [x / denom for x in nums]
#     choice = random.choices(list(options.keys()), weights=choice_probs)[0]
#     return choice


# def solve_cryptogram(text, tokenizer, max_epochs=250, temp=0.05):
#     doc = prep.Doc(text)
#     best_nll = tokenizer.calc_nll(doc.text)
#     best_swap = ('a', 'a')
#     epoch = 0
#     improving = True
#     while True and epoch < max_epochs:
#         improving = False
#         print('*', end='')
#         options = dict()
#         for letter1 in doc.letters:
#             for letter2 in LETTERS - set(letter1):
#                 swap = tuple(sorted([letter1, letter2]))
#                 if swap in options:
#                     continue
#                 new_text = doc.swap(*swap)
#                 nll = tokenizer.calc_nll(new_text)
#                 options[swap] = -nll
#                 if nll < best_nll:
#                     print(nll)
#                     best_swap = swap
#                     best_nll = nll
#                     improving = True
#         chosen_swap = softmax(options, temp)
#         doc = prep.Doc(doc.swap(*chosen_swap))
#         epoch += 1
#         # print(f'best of epoch: {best_nll:0.5g}')
#     print(f'\nfinal best ({epoch} epochs): {best_nll:0.5g}')
#     return doc.text, options

In [544]:
text = doc2.text
text = 'Rbo rpktigo vcrb bwucja wj kloj hcjd, km sktpqo, cq rbwr loklgo vcgg cjqcqr kj skhcja wgkja wjd rpycja rk ltr rbcjaq cj cr. -- Roppy Lpwrsborr'
text = 'This is the story of a girl. Who cried a river and drowned the whole world. And while she looked so sad in photographs, I absolutely loved her when she smiled.'
doc = prep.Doc(text)
doc.scramble(25)
doc.text

'mizc zc mie cmbuv bt l azuy. wib ruzed l uzseu lnd dubwned mie wibye wbuyd. lnd wizye cie ybbged cb cld zn xibmbaulxic, z lfcbykmeyv ybsed ieu wien cie cjzyed.'

In [545]:
# text, options = solve_cryptogram(doc.text, tokenizer)
text = solve_cryptogram(doc.text, tokenizer)
print(text)

*misc sc mie cmbuv bt l asuy. wib rused l uszeu lnd dubwned mie wibye wbuyd. lnd wisye cie ybbged cb cld sn xibmbaulxic, s lfcbykmeyv ybzed ieu wien cie cjsyed.
best of epoch: 23.261
*misc sc mie cmouv ot l asuy. wio rused l uszeu lnd duowned mie wioye wouyd. lnd wisye cie yooged co cld sn xiomoaulxic, s lfcoykmeyv yozed ieu wien cie cjsyed.
best of epoch: 22.696
*misc sc mie cmouv ot y asul. wio rused y uszeu ynd duowned mie wiole would. ynd wisle cie looged co cyd sn xiomoauyxic, s yfcolkmelv lozed ieu wien cie cjsled.
best of epoch: 22.135
*misc sc mie cmouv ot a ysul. wio rused a uszeu and duowned mie wiole would. and wisle cie looged co cad sn xiomoyuaxic, s afcolkmelv lozed ieu wien cie cjsled.
best of epoch: 21.582
*mist st mie tmouv oc a ysul. wio rused a uszeu and duowned mie wiole would. and wisle tie looged to tad sn xiomoyuaxit, s aftolkmelv lozed ieu wien tie tjsled.
best of epoch: 21.338
*mhst st mhe tmouv oc a ysul. who rused a uszeu and duowned mhe whole would. and whsl

In [517]:
from collections import Counter

In [518]:
Counter(['a', 'b', 'c', 'c'])

Counter({'a': 1, 'b': 1, 'c': 2})

In [1876]:
def softmax(options, temp=1):
    nums = [exp(x / temp) for x in options.values()]
    denom = sum(nums)
    choice_probs = [x / denom for x in nums]
    choice = random.choices(list(options.keys()), weights=choice_probs)[0]
    return choice


# options = {'a': 1, 'b': 2}
softmax(options, temp=0.05)

4.608693758495905e-247
0.43184276106555297


('b', 'q')

In [1775]:
softmax(options)

[4.5397868702434395e-05, 0.9999546021312976]


'b'